# **\[Essays\]** 강화학습(1) – bandit

신록예찬  
2023-08-23

# 환경셋팅

`-` 설치

``` python
!pip install -q swig
!pip install gymnasium
!pip install gymnasium[box2d]
```

# imports

In [5]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt

# Intro

`-` 강화학습(대충설명): 어떠한 “(게임)환경”이 있을때 거기서 “뭘 할지”를
학습하는 과업

`-` 딥마인드: breakout $\to$ 알파고

<https://www.youtube.com/watch?v=TmPfTpjtdgg>

`-` 강화학습의 미래? (이거 잘하면 먹고 살 수 있을까?)

# Game1: 벤딧문제

`-` 문제설명: 두개의 버튼이 있다. 버튼0을 누르면 1의 보상을, 버튼1를
누르면 100의 보상을 준다고 가정

`-` 어떤 행동을 해야할까? –\> ?? 아는게없음 –\> 일단 “아무거나” 눌러보자

In [6]:
action_space = ['button0','button1']
action = np.random.choice(action_space)
action

`-` 보상은 아래와 같은 방식으로 받을 것이다.

In [7]:
if action == 'button0': 
    reward = 1 
else:
    reward = 100

`-` 아무거나 10번 버튼을 눌러보면 다음과 같은 결과가 나온다.

In [8]:
for _ in range(10):
    action = np.random.choice(action_space)
    if action == 'button0': 
        reward = 1 
    else:
        reward = 100
    print(action,reward)

button1 100
button0 1
button1 100
button1 100
button1 100
button1 100
button0 1
button0 1
button1 100
button1 100

`-` 깨달았음: `button0`을 누르면 1점을 받고, `button1`을 누르면 100점을
받는 “환경”이구나? $\to$ `button1`을 누르는 “동작”을 해야하는
상황이구나?

-   여기에서 $\to$ 의 과정을 체계화 시킨 학문이 강화학습이다.

In [9]:
action_space = ['button0','button1']
for _ in range(10):
    #action = np.random.choice(action_space) # 무지한자의 행동 (찍어)
    action = action_space[1] # 깨달은자의 행동 (button1을 눌러)
    if action == 'button0': 
        reward = 1 
    else:
        reward = 100
    print(action,reward)

button1 100
button1 100
button1 100
button1 100
button1 100
button1 100
button1 100
button1 100
button1 100
button1 100

`-` 강화학습: 환경의 이해 $\to$ 뭘 해야 하는지 깨달음

**위의 과정이 잘 되었다는 의미로 사용하는 문장들**

-   강화학습이 성공적으로 잘 되었다.
-   환경이 해결되었다.
-   에이전트가 환경의 과제를 완료했다.
-   에이전트가 환경에서 성공적으로 학습했다.
-   에이전트가 올바른 행동을 학습했다.
-   게임 클리어 (비공식)

`-` 환경이 해결되었는지 나타내는 지표를 정해야겠다 $\to$ 주어진 상황을
게임처럼 이해하고, 게임의 클리어조건을 설정

-   첫 생각: `button1`을 누르면 클리어 아니야?
-   두번째 생각: 아니지? 우연히 누를수도 있잖아.
-   게임클리어조건: 최근 20번의 보상이 1900 이상이면 게임이 클리어
    되었다고 보자![1]

`-` 무지한자 – 게임을 클리어 할 수 없다.

[1] `button1`을 눌러야하는건 맞지만 20번에 실수한번정도는 눈감아 주자

In [10]:
action_space = ['button0','button1']
rewards = []
for i in range(50): # 1000번해도 못깸
    action = np.random.choice(action_space) # 무지한자의 행동 (찍어)
    #action = action_space[1] # 깨달은자의 행동 (button1을 눌러)
    if action == 'button0': 
        reward = 1 
        rewards.append(reward)
    else:
        reward = 100
        rewards.append(reward)
    print(
        f"n_try = {i+1}\t"
        f"action = {action}\t"
        f"reward = {reward}\t"
        f"rwd20 = {sum(rewards[-20:])}"
    )
    if np.sum(rewards[-20:])>1900:
        break
    

n_try = 1   action = button1    reward = 100    rwd20 = 100
n_try = 2   action = button1    reward = 100    rwd20 = 200
n_try = 3   action = button1    reward = 100    rwd20 = 300
n_try = 4   action = button0    reward = 1  rwd20 = 301
n_try = 5   action = button1    reward = 100    rwd20 = 401
n_try = 6   action = button1    reward = 100    rwd20 = 501
n_try = 7   action = button0    reward = 1  rwd20 = 502
n_try = 8   action = button1    reward = 100    rwd20 = 602
n_try = 9   action = button0    reward = 1  rwd20 = 603
n_try = 10  action = button1    reward = 100    rwd20 = 703
n_try = 11  action = button0    reward = 1  rwd20 = 704
n_try = 12  action = button0    reward = 1  rwd20 = 705
n_try = 13  action = button0    reward = 1  rwd20 = 706
n_try = 14  action = button1    reward = 100    rwd20 = 806
n_try = 15  action = button1    reward = 100    rwd20 = 906
n_try = 16  action = button0    reward = 1  rwd20 = 907
n_try = 17  action = button1    reward = 100    rwd20 = 1007
n_try =

`-` 깨달은자 – 게임 클리어

In [11]:
action_space = ['button0','button1']
rewards = []
for i in range(50): # 1000번해도 못깸
    #action = np.random.choice(action_space) # 무지한자의 행동 (찍어)
    action = action_space[1] # 깨달은자의 행동 (button1을 눌러)
    if action == 'button0': 
        reward = 1 
        rewards.append(reward)
    else:
        reward = 100
        rewards.append(reward)
    print(
        f"n_try = {i+1}\t"
        f"action = {action}\t"
        f"reward = {reward}\t"
        f"rwd20 = {sum(rewards[-20:])}"
    )
    if np.sum(rewards[-20:])>1900:
        break

n_try = 1   action = button1    reward = 100    rwd20 = 100
n_try = 2   action = button1    reward = 100    rwd20 = 200
n_try = 3   action = button1    reward = 100    rwd20 = 300
n_try = 4   action = button1    reward = 100    rwd20 = 400
n_try = 5   action = button1    reward = 100    rwd20 = 500
n_try = 6   action = button1    reward = 100    rwd20 = 600
n_try = 7   action = button1    reward = 100    rwd20 = 700
n_try = 8   action = button1    reward = 100    rwd20 = 800
n_try = 9   action = button1    reward = 100    rwd20 = 900
n_try = 10  action = button1    reward = 100    rwd20 = 1000
n_try = 11  action = button1    reward = 100    rwd20 = 1100
n_try = 12  action = button1    reward = 100    rwd20 = 1200
n_try = 13  action = button1    reward = 100    rwd20 = 1300
n_try = 14  action = button1    reward = 100    rwd20 = 1400
n_try = 15  action = button1    reward = 100    rwd20 = 1500
n_try = 16  action = button1    reward = 100    rwd20 = 1600
n_try = 17  action = button1    r

# 수정1: `action_space`의 수정

In [12]:
action_space =  gym.spaces.Discrete(2)
action_space

`-` 좋은점1: sample

In [13]:
for _ in range(10):
    print(action_space.sample())

0
1
0
1
1
0
0
0
0
1

`-` 좋은점2: in

In [14]:
0 in action_space

In [15]:
1 in action_space

In [16]:
2 in action_space

`-` 코드 1차수정

In [17]:
action_space = gym.spaces.Discrete(2)
rewards = []
for i in range(50): # 1000번해도 못깸
    action = action_space.sample() # 무지한자의 행동 (찍어)
    #action = 1 # 깨달은자의 행동 (button1을 눌러)
    if action == 0: 
        reward = 1 
        rewards.append(reward)
    else:
        reward = 100
        rewards.append(reward)
    print(
        f"n_try = {i+1}\t"
        f"action = {action}\t"
        f"reward = {reward}\t"
        f"rwd20 = {sum(rewards[-20:])}"
    )
    if np.sum(rewards[-20:])>1900:
        break

n_try = 1   action = 0  reward = 1  rwd20 = 1
n_try = 2   action = 0  reward = 1  rwd20 = 2
n_try = 3   action = 0  reward = 1  rwd20 = 3
n_try = 4   action = 0  reward = 1  rwd20 = 4
n_try = 5   action = 0  reward = 1  rwd20 = 5
n_try = 6   action = 1  reward = 100    rwd20 = 105
n_try = 7   action = 1  reward = 100    rwd20 = 205
n_try = 8   action = 1  reward = 100    rwd20 = 305
n_try = 9   action = 1  reward = 100    rwd20 = 405
n_try = 10  action = 0  reward = 1  rwd20 = 406
n_try = 11  action = 0  reward = 1  rwd20 = 407
n_try = 12  action = 1  reward = 100    rwd20 = 507
n_try = 13  action = 0  reward = 1  rwd20 = 508
n_try = 14  action = 0  reward = 1  rwd20 = 509
n_try = 15  action = 0  reward = 1  rwd20 = 510
n_try = 16  action = 0  reward = 1  rwd20 = 511
n_try = 17  action = 1  reward = 100    rwd20 = 611
n_try = 18  action = 0  reward = 1  rwd20 = 612
n_try = 19  action = 0  reward = 1  rwd20 = 613
n_try = 20  action = 1  reward = 100    rwd20 = 713
n_try = 21  action = 0

# 수정2: Env 클래스

`-` env 생성

In [18]:
class Bandit:
    def step(self,action):
        if action == 0: 
            return 1
        else:
            return 100

In [19]:
env = Bandit()
rewards = []
for i in range(50): # 1000번해도 못깸
    #action = action_space.sample() # 무지한자의 행동 (찍어)
    action = 1 # 깨달은자의 행동 (button1을 눌러)
    reward = env.step(action)
    rewards.append(reward)
    print(
        f"n_try = {i+1}\t"
        f"action = {action}\t"
        f"reward = {reward}\t"
        f"rwd20 = {sum(rewards[-20:])}"
    )
    if np.sum(rewards[-20:])>1900:
        break

n_try = 1   action = 1  reward = 100    rwd20 = 100
n_try = 2   action = 1  reward = 100    rwd20 = 200
n_try = 3   action = 1  reward = 100    rwd20 = 300
n_try = 4   action = 1  reward = 100    rwd20 = 400
n_try = 5   action = 1  reward = 100    rwd20 = 500
n_try = 6   action = 1  reward = 100    rwd20 = 600
n_try = 7   action = 1  reward = 100    rwd20 = 700
n_try = 8   action = 1  reward = 100    rwd20 = 800
n_try = 9   action = 1  reward = 100    rwd20 = 900
n_try = 10  action = 1  reward = 100    rwd20 = 1000
n_try = 11  action = 1  reward = 100    rwd20 = 1100
n_try = 12  action = 1  reward = 100    rwd20 = 1200
n_try = 13  action = 1  reward = 100    rwd20 = 1300
n_try = 14  action = 1  reward = 100    rwd20 = 1400
n_try = 15  action = 1  reward = 100    rwd20 = 1500
n_try = 16  action = 1  reward = 100    rwd20 = 1600
n_try = 17  action = 1  reward = 100    rwd20 = 1700
n_try = 18  action = 1  reward = 100    rwd20 = 1800
n_try = 19  action = 1  reward = 100    rwd20 = 1900
n_

# 수정3: Agent 클래스

`-` Agent 클래스를 만들자 (액션을 하고, 환경에서 받은 reward를
간직하도록)

In [20]:
class Agent1:
    def __init__(self):
        self.action_space = gym.spaces.Discrete(2)
        self.action = None
        self.reward = None
        self.actions = []
        self.rewards = []
    def act(self):
        # self.action = self.action_space.sample() # 무지한 자
        self.action = 1 # 깨달은 자 
    def save_experience(self): 
        self.actions.append(self.action)
        self.rewards.append(self.reward)

— 대충 아래와 같은 느낌으로 돌아가요 —

**시점0**: init

In [21]:
env = Bandit()
agent = Agent1()

In [22]:
agent.action, agent.reward

**시점1**: agent \>\> env

In [23]:
agent.act()

In [24]:
agent.action, agent.reward

In [25]:
env.agent_action = agent.action 

**시점2**: agent \<\< env

In [26]:
agent.reward = env.step(env.agent_action) 
# agent.reward = env.step(agent.action) 도 같은 결과를 주는 코드임!!

In [27]:
agent.action, agent.reward, env.agent_action

In [28]:
agent.actions, agent.rewards

In [29]:
agent.save_experience()

In [30]:
agent.actions, agent.rewards

— 전체코드 —

In [31]:
env = Bandit()
agent = Agent1()
for i in range(50):
    ## 1. 본질적인 코드
    # step1: agent >> env 
    agent.act()
    env.agent_action = agent.action 
    # step2: agnet << env 
    agent.reward = env.step(env.agent_action) 
    agent.save_experience()
    
    ## 2. 비본질적 코드 
    print(
        f"n_try = {i+1}\t"
        f"action = {agent.action}\t"
        f"reward = {agent.reward}\t"
        f"rwd20 = {sum(agent.rewards[-20:])}"
    )
    if np.sum(agent.rewards[-20:])>1900:
        break

n_try = 1   action = 1  reward = 100    rwd20 = 100
n_try = 2   action = 1  reward = 100    rwd20 = 200
n_try = 3   action = 1  reward = 100    rwd20 = 300
n_try = 4   action = 1  reward = 100    rwd20 = 400
n_try = 5   action = 1  reward = 100    rwd20 = 500
n_try = 6   action = 1  reward = 100    rwd20 = 600
n_try = 7   action = 1  reward = 100    rwd20 = 700
n_try = 8   action = 1  reward = 100    rwd20 = 800
n_try = 9   action = 1  reward = 100    rwd20 = 900
n_try = 10  action = 1  reward = 100    rwd20 = 1000
n_try = 11  action = 1  reward = 100    rwd20 = 1100
n_try = 12  action = 1  reward = 100    rwd20 = 1200
n_try = 13  action = 1  reward = 100    rwd20 = 1300
n_try = 14  action = 1  reward = 100    rwd20 = 1400
n_try = 15  action = 1  reward = 100    rwd20 = 1500
n_try = 16  action = 1  reward = 100    rwd20 = 1600
n_try = 17  action = 1  reward = 100    rwd20 = 1700
n_try = 18  action = 1  reward = 100    rwd20 = 1800
n_try = 19  action = 1  reward = 100    rwd20 = 1900
n_

-   위의 코드를 거의 뼈대로 사용합니당!

# 학습과정 포함

`-` Game1에 대한 생각:

-   사실 강화학습은 “환경을 이해 $\to$ 뭘 해야 하는지 깨달음” 의
    과정에서 화살표를 수식화 하는 과정이다.
-   지금까지 살펴보면 Game1은 환경(env)을 이해하는 순간 에이전트가
    최적의 행동(action)[1]을 직관적으로 결정하였으므로 학습의 과정이
    포함되었다고 볼 수 없다.

`-` 지금까지의 코드 복습

1.  클래스를 선언하는 부분
    -   `Env`클래스의 선언
    -   `Agent`클래스의 선언
2.  환경과 에이전트를 인스턴스화 (초기화)
3.  for loop를 반복하며 게임을 진행
    -   메인코드: (1) agent \>\> env (2) agent \<\< env (3) 데이터저장
    -   비본질적코드: 학습과정 display, 학습종료조건

`-` 앞으로 구성할 코드의 형태: 에이전트가 데이터를 보고 스스로
`button1`을 눌러야 한다는 것을 추론하면 좋겠음

1.  클래스를 선언하는 부분
    -   `Env`클래스의 선언
    -   **`Agent`클래스의 선언**: **학습의 과정 포함 $\to$ act함수
        수정**
2.  환경과 에이전트를 인스턴스화 (초기화)
3.  for loop를 반복하며 게임을 진행
    -   메인코드: (1) agent \>\> env (2) agent \<\< env (3) 데이터저장
        **(4) 학습**
    -   비본질적코드: 학습과정 display, 학습종료조건

`-` 에이전트가 학습을 어떻게 하는가?[2]

-   $\pi(0) = \frac{Q_t(1)}{Q_t(1)+Q_t(0)}$
-   $\pi(1) = \frac{Q_t(0)}{Q_t(1)+Q_t(0)}$

여기에서 각각의 기호는 아래를 의미한다.

-   $\pi(0)$: 에이전트가 action = 0 을 할 확률, 즉 에이전트가
    `button0`을 누를 확률
-   $\pi(1)$: 에이전트가 action = 1 을 할 확률, 즉 에이전트가
    `button1`을 누를 확률
-   $Q_t(0)$: ($t$시점까지 파악한) action = 0 을 하였을 경우 에이전트가
    환경으로 받은 보상의 평균값
-   $Q_t(1)$: ($t$시점까지 파악한) action = 1 을 하였을 경우 에이전트가
    환경으로 받은 보상의 평균값

`-` 걱정: $t=0$이면 어쩌지? $t=1$이면 어쩌지?.. $\to$ 잡기술1: 일정
시간동안은 랜덤액션을 하면서 데이터를 쌓자.

`-` 쌓은 데이터를 바탕으로 환경을 이해하고 action을 뽑는 코드

[1] `button1`을 누른다

[2] 행동을 이런식으로 하도록 “전략(=정책)”을 설정하는 것은 상식적이다.
그렇지만 유일한 해결책은 아님!!

In [33]:
agent.actions = [0,0,1,0,0,1,1,1,1,0]
agent.actions 

In [34]:
agent.rewards = [1,0.9,105,1.2,1,95,100,101,90,1]
agent.rewards

In [45]:
actions = np.array(agent.actions)
rewards = np.array(agent.rewards)

In [46]:
q0= rewards[actions==0].mean()
q1= rewards[actions==1].mean()

In [47]:
agent.q = np.array([q0,q1])
agent.q

In [48]:
action = np.random.choice([0,1],p=agent.q/agent.q.sum())
action

`-` 코드를 구현해보자.

In [30]:
# 게임을 푸는 전략을 바꾸는 것이지 게임자체를 바꿀게 아니니까 수정할게 없죠?
class Bandit():
    def step(self,action):
        if action == 0: 
            return 1
        else:
            return 100

In [64]:
# learn 추가
class Agent:
    def __init__(self):
        self.action_space = gym.spaces.Discrete(2)
        self.action = None
        self.reward = None
        self.actions = []
        self.rewards = []
        self.n_experiences = 0
        self.q = np.array([0,0])
    def act(self):
        if self.n_experiences < 30:
            self.action = self.action_space.sample()
        else:
            self.action = np.random.choice([0, 1], p=self.q/self.q.sum())
    def save_experience(self): 
        self.actions.append(self.action)
        self.rewards.append(self.reward)
        self.n_experiences = self.n_experiences + 1 
    def learn(self):
        if self.n_experiences < 30: 
            pass 
        else:
            actions = np.array(self.actions)
            rewards = np.array(self.rewards)
            q0= rewards[actions==0].mean() 
            q1= rewards[actions==1].mean()        
            self.q = np.array([q0,q1])

In [65]:
env = Bandit()
agent = Agent()
for i in range(60):
    ## 본질적인 코드
    # step1: agent >> env 
    agent.act()
    env.agent_action = agent.action 
    # step2: agnet << env 
    agent.reward = env.step(env.agent_action) 
    agent.save_experience()
    # step3: learn
    agent.learn()
    
    ## 비본질적 코드 
    print(
        f"n_try = {agent.n_experiences}\t"
        f"action = {agent.action}\t"
        f"reward = {agent.reward}\t"
        f"rwd20 = {np.sum(agent.rewards[-20:])}\t"
        f"q = {agent.q}"
    )
    if np.sum(agent.rewards[-20:])>1900:
        break

n_try = 1   action = 1  reward = 100    rwd20 = 100 q = [0 0]
n_try = 2   action = 1  reward = 100    rwd20 = 200 q = [0 0]
n_try = 3   action = 1  reward = 100    rwd20 = 300 q = [0 0]
n_try = 4   action = 0  reward = 1  rwd20 = 301 q = [0 0]
n_try = 5   action = 0  reward = 1  rwd20 = 302 q = [0 0]
n_try = 6   action = 1  reward = 100    rwd20 = 402 q = [0 0]
n_try = 7   action = 0  reward = 1  rwd20 = 403 q = [0 0]
n_try = 8   action = 1  reward = 100    rwd20 = 503 q = [0 0]
n_try = 9   action = 0  reward = 1  rwd20 = 504 q = [0 0]
n_try = 10  action = 0  reward = 1  rwd20 = 505 q = [0 0]
n_try = 11  action = 1  reward = 100    rwd20 = 605 q = [0 0]
n_try = 12  action = 1  reward = 100    rwd20 = 705 q = [0 0]
n_try = 13  action = 1  reward = 100    rwd20 = 805 q = [0 0]
n_try = 14  action = 1  reward = 100    rwd20 = 905 q = [0 0]
n_try = 15  action = 1  reward = 100    rwd20 = 1005    q = [0 0]
n_try = 16  action = 0  reward = 1  rwd20 = 1006    q = [0 0]
n_try = 17  action = 0  